# Logistique de la vaccination Covid en Belgique
### Groupe 41
**Aigret Julien 8343-13-00,  
Suetens Lucas,  
Zaleschi Florina**
## Partie 1
### Question I.1
#### Initialisation des données

In [1]:
positive_inf=float('inf')
negative_inf=-float('inf')
import math
#import sys
#!{sys.executable} -m pip install mip
from mip import *
import matplotlib.pyplot as plt

nb_days = 350
deliveries = [[7, 80000],
[14, 80000],
[21, 60000],
[28, 60000],
[35, 40000],
[42, 40000],
[49, 40000],
[56, 40000],
[63, 60000],
[70, 60000],
[77, 60000],
[84, 60000],
[91, 80000],
[98, 80000],
[105, 80000],
[112, 80000],
[119, 100000],
[126, 100000],
[133, 100000],
[140, 100000],
[147, 100000],
[154, 100000],
[161, 100000],
[168, 100000],
[175, 120000],
[182, 120000],
[189, 120000],
[196, 120000],
[203, 120000],
[210, 120000],
[217, 120000],
[224, 120000],
[231, 120000],
[238, 120000],
[245, 120000],
[252, 120000],
[259, 150000],
[266, 150000],
[273, 150000],
[280, 150000],
[287, 150000],
[294, 150000],
[301, 150000],
[308, 150000],
[315, 150000],
[322, 150000],
[329, 150000],
[336, 150000],
[343, 150000],
[350, 150000]]

population = {'Y': 3778123, 'A': 2846993, 'S': 2790883, 'O': 1390502, 'C': 111533}

def frac_to_get_ill(classe, jour):
    if classe == 'Y':
        return 0.000298 * (1/5 + math.sin(jour/50-1/5)**2)
    if classe == 'A':
        return 0.000301 * (1/5 + math.sin(jour/50)**2)
    if classe == 'S':
        return 0.000204 * (1/5 + math.sin(jour/50-1/5)**2)
    if classe == 'O':
        return 0.000209 * (1/5 + math.sin(jour/50-2/5)**2)
    if classe == 'C':
        return 0.000329 * (1/5 + math.sin(jour/50-2/5)**2)

def frac_to_die(classe, jour):
    if classe == 'Y':
        return 0.000100 * (6/5-jour/1000)
    if classe == 'A':
        return 0.000400 * (6/5-jour/1000)
    if classe == 'S':
        return 0.005790 * (6/5-jour/1000)
    if classe == 'O':
        return 0.027179 * (6/5-jour/1000)
    if classe == 'C':
        return 0.150000 * (6/5-jour/1000)
    
to_be_vaccined =  {'Y' : 0.3, 'A' : 0.6, 'S' : 0.7, 'O' : 0.9, 'C' : 0.9}
transport_cost = [0]
max_delivery = [positive_inf]
vaccination_cost = [15]
max_vaccination = [14646]
stock_capa = [0]
budget= 100 * 10**6
classes = ['Y', 'A', 'S', 'O', 'C']

#### Description et résolution du modèle

In [2]:
"""
Variables :
- susc['classe'][t] : population de la classe 'classe' susceptible de tomber malade (dictionnaire) au jour t
- mort['classe'][t] : population de la classe 'classe' morte (dictionnaire) au jour t
fonction obj : min sum(d['classes'][350])
contraintes :
au jour t, on vaccine 
"""

m = Model()

susc = dict()
mort = dict()
vaccin_delivered = dict()
vaccine_given = dict()

for C in classes:
    susc[C] = [m.add_var(lb=0) for _ in range(nb_days+1)]
    mort[C] = [m.add_var(lb=0) for _ in range(nb_days+1)]
    vaccine_given[C] = [m.add_var(lb=0) for _ in range(nb_days+1)]

vaccin_delivered = [m.add_var(lb=0) for _ in range(nb_days+1)]

m.objective = minimize(xsum(mort[C][350] for C in classes))

for C in classes:
    m += susc[C][0] == population[C]
    m += mort[C][0] == 0

livraisons = [0 for _ in range(nb_days+1)]
for [day, doses] in deliveries:
    livraisons[day] = doses

for t in range(nb_days):
    m += vaccin_delivered[t+1] <= livraisons[t]
    
for t in range(nb_days+1):
    m += vaccin_delivered[t] <= max_delivery[0]
    m += xsum(vaccine_given[C][t] for C in classes) <= vaccin_delivered[t] #""" + stock, pas pour cette question"""
    m += xsum(vaccine_given[C][t] for C in classes) <= max_vaccination[0]
    
for C in classes:
    for t in range(nb_days):
        m += vaccine_given[C][t] <= susc[C][t] * to_be_vaccined[C]
        m += mort[C][t+1] == mort[C][t] + susc[C][t] * frac_to_get_ill(C,t) * frac_to_die(C,t)
        m += susc[C][t+1] == susc[C][t] - susc[C][t]*frac_to_get_ill(C,t) - vaccine_given[C][t]

m += xsum(vaccine_given[C][t] for t in range(nb_days+1) for C in classes)*15 <= budget

status = m.optimize()
if status == OptimizationStatus.OPTIMAL:
    print('optimal solution cost {} found'.format(m.objective_value))
elif status == OptimizationStatus.FEASIBLE:
    print('sol.cost {} found, best possible: {}'.format(m.objective_value, m.objective_bound))
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
    print('no feasible solution found, lower bound is: {}'.format(m.objective_bound))

optimal solution cost 2392.7372478409043 found


In [3]:
total_deaths = m.objective_value
used_budget = xsum(15 * vaccine_given[C][t] for C in classes for t in range(nb_days+1)).x
people_vaccined = xsum(vaccine_given[C][t] for C in classes for t in range(nb_days+1)).x
print(f"Morts dans ce modèle : {total_deaths:.3f}, budget utilisé : {used_budget:.3f}€ ({100*used_budget/budget:.3f}%), vaccin administré : {people_vaccined}")

Morts dans ce modèle : 2392.737, budget utilisé : 10984500.000€ (10.985%), vaccin administré : 732300.0


In [4]:
for days in range(nb_days):
    for C in classes:
        if vaccine_given[C][days+1].x != 0:
            print(f"Jour {days+1}, classe {C} : {vaccine_given[C][days+1].x}")
print(sum(vaccine_given[C][t].x for t in range(nb_days+1) for C in classes))
for C in classes:
    print(f"Classe {C} : susceptible {susc[C][350].x}, morts {mort[C][350].x}")

Jour 8, classe C : 14646.000000000002
Jour 15, classe C : 14646.0
Jour 22, classe C : 14645.999999999998
Jour 29, classe C : 14646.0
Jour 36, classe C : 14646.0
Jour 43, classe C : 14646.0
Jour 50, classe O : 6751.188380755475
Jour 50, classe C : 7894.811619244524
Jour 57, classe O : 13858.777495456188
Jour 57, classe C : 787.2225045438126
Jour 64, classe O : 14567.559586702288
Jour 64, classe C : 78.4404132977134
Jour 71, classe O : 14638.18974207226
Jour 71, classe C : 7.810257927739544
Jour 78, classe O : 14645.222867301229
Jour 78, classe C : 0.7771326987702979
Jour 85, classe O : 14645.922718859756
Jour 85, classe C : 0.07728114024405597
Jour 92, classe O : 14645.992318177921
Jour 92, classe C : 0.007681822077885109
Jour 99, classe O : 14645.999236611073
Jour 99, classe C : 0.0007633889251099334
Jour 106, classe O : 14645.999924141299
Jour 106, classe C : 7.585869807278369e-05
Jour 113, classe O : 14645.999992460691
Jour 113, classe C : 7.539309852276097e-06
Jour 120, classe O : 1

### Question I.2
#### Initialisation des données

In [5]:
import pandas as pd
filepath = 'Données-v1.2/'
centres = pd.read_csv(filepath+'Centres_vaccination.csv', index_col=0)
fractions = pd.read_csv(filepath+'Fraction_malade_province.csv', index_col=0)
populations = pd.read_csv(filepath+'Population_province.csv', index_col=0)

prov = centres.index # liste des noms des centres

def frac_to_get_ill(classe, jour, province):
    if classe == 'Y':
        return fractions[province]['young'] * (1/5 + math.sin(jour/50-1/5)**2)
    if classe == 'A':
        return fractions[province]['adult'] * (1/5 + math.sin(jour/50)**2)
    if classe == 'S':
        return fractions[province]['senior'] * (1/5 + math.sin(jour/50-1/5)**2)
    if classe == 'O':
        return fractions[province]['old'] * (1/5 + math.sin(jour/50-2/5)**2)
    if classe == 'C':
        return fractions[province]['centanarian'] * (1/5 + math.sin(jour/50-2/5)**2)
    
def get_population(province, classe):
    if classe == 'Y':
        return int(populations.loc[(populations.index == province) & (populations['Tranche'] == 'young'), 'Population'])
    if classe == 'A':
        return int(populations.loc[(populations.index == province) & (populations['Tranche'] == 'adult'), 'Population'])
    if classe == 'S':
        return int(populations.loc[(populations.index == province) & (populations['Tranche'] == 'senior'), 'Population'])
    if classe == 'O':
        return int(populations.loc[(populations.index == province) & (populations['Tranche'] == 'old'), 'Population'])
    if classe == 'C':
        return int(populations.loc[(populations.index == province) & (populations['Tranche'] == 'centanarian'), 'Population'])

max_delivery = dict()
max_vaccination = dict()
max_stock = dict()
cost_delivery = dict()
cost_vaccination = dict()
cost_stock = dict()
for p in prov:
    max_delivery[p] = int(centres.loc[(centres.index == p), 'CapaciteTransport'])
    max_vaccination[p] = int(centres.loc[(centres.index == p), 'CapaciteVaccination'])
    max_stock[p] = int(centres.loc[(centres.index == p), 'CapaciteStockage'])
    cost_delivery[p] = int(centres.loc[(centres.index == p), 'CoutTransport'])
    cost_vaccination[p] = int(centres.loc[(centres.index == p), 'CoutDose'])
    cost_stock[p] = int(centres.loc[(centres.index == p), 'CoutStockage'])


nb_days = 380
first_del = deliveries[0][0]

#### Description et résolution du modèle

In [6]:
m = Model()

susc = dict()
mort = dict()
vaccin_delivered = dict()
vaccine_given = dict()
vaccin_delivered = dict()
stock = dict()

for p in prov:
    susc[p] = dict()
    mort[p] = dict()
    vaccine_given[p] = dict()
    for C in classes:
        susc[p][C] = [m.add_var(lb=0) for _ in range(nb_days+1)]
        mort[p][C] = [m.add_var(lb=0) for _ in range(nb_days+1)]
        vaccine_given[p][C] = [m.add_var(lb=0) for _ in range(nb_days+1)]
    stock[p] = [m.add_var(lb=0) for _ in range(nb_days+1)]
    vaccin_delivered[p] = [m.add_var(lb=0) for _ in range(nb_days+1)]

m.objective = minimize(xsum(mort[p][C][350] for C in classes for p in prov))
for p in prov:
    m += stock[p][0] == 0
    for t in range(nb_days):
        m += stock[p][t+1] == stock[p][t] + vaccin_delivered[p][t] - xsum(vaccine_given[p][C][t] for C in classes)
    for C in classes:
        m += susc[p][C][0] == get_population(p, C)
        m += mort[p][C][0] == 0

livraisons = [0 for _ in range(nb_days+1)]
for [day, doses] in deliveries:
    livraisons[day] = doses

for t in range(nb_days):
    m += xsum(vaccin_delivered[p][t+1] for p in prov) <= livraisons[t]

for p in prov:
    m += vaccin_delivered[p][0] == 0
    
    for t in range(nb_days+1):
        m += stock[p][t] <= max_stock[p]
        m += stock[p][t] >= 0
        m += vaccin_delivered[p][t] <= max_delivery[p]
        m += vaccin_delivered[p][t] >= 0
        m += xsum(vaccine_given[p][C][t] for C in classes) <= vaccin_delivered[p][t] + stock[p][t]
        m += xsum(vaccine_given[p][C][t] for C in classes) <= max_vaccination[p]

    for C in classes:
        for t in range(nb_days):
            m += vaccine_given[p][C][t] <= susc[p][C][t] * to_be_vaccined[C]
            m += vaccine_given[p][C][t] >= 0
            m += mort[p][C][t+1] == mort[p][C][t] + susc[p][C][t] * frac_to_get_ill(C,t,p) * frac_to_die(C,t)
            m += susc[p][C][t+1] == susc[p][C][t] - susc[p][C][t]*frac_to_get_ill(C,t,p) - vaccine_given[p][C][t]

m += xsum(cost_vaccination[p] * vaccine_given[p][C][t] for C in classes for t in range(nb_days+1) for p in prov) + xsum(vaccin_delivered[p][t] * cost_delivery[p] + stock[p][t] * cost_stock[p] for t in range(nb_days+1) for p in prov) <= budget

status = m.optimize()
if status == OptimizationStatus.OPTIMAL:
    print('optimal solution cost {} found'.format(round(m.objective_value), 6))
elif status == OptimizationStatus.FEASIBLE:
    print('sol.cost {} found, best possible: {}'.format(round(m.objective_value, 6), round(m.objective_bound, 6)))
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
    print('no feasible solution found, lower bound is: {}'.format(round(m.objective_bound, 6)))

optimal solution cost 983 found


#### Analyse du modèle

In [7]:
total_deaths = round(m.objective_value,6)
used_budget = round(xsum(cost_vaccination[p] * vaccine_given[p][C][t] for C in classes for t in range(nb_days+1) for p in prov).x + xsum(vaccin_delivered[p][t] * cost_delivery[p] + stock[p][t] * cost_stock[p] for t in range(nb_days+1) for p in prov).x,6)
people_vaccined = round(xsum(vaccine_given[p][C][t] for p in prov for C in classes for t in range(nb_days+1)).x,6)
vaccine_given_per_day = dict()
for p in prov:
    vaccine_given_per_day[p] = []
    for t in range(nb_days+1):
        vaccine_given_per_day[p].append(round(xsum(vaccine_given[p][C][t] for C in classes).x, 6))
print(f"Morts dans ce modèle : {total_deaths:.3f}, budget utilisé : {used_budget:.3f}€ ({used_budget/budget*100:.3f}%), vaccins administrés : {people_vaccined}")
for t in range(nb_days):
    if t > 6:
        if round(xsum(vaccin_delivered[p][t+1] for p in prov).x, 6) == livraisons[t] and livraisons[t] != 0:
            count = 0
            for p in prov:
                count += min(round(max_delivery[p] - vaccin_delivered[p][t+1].x, 6), round(max_stock[p] - stock[p][t+2].x, 6) + round(max_vaccination[p] - vaccine_given_per_day[p][t+1], 6))
            print(f"|{t}|{round(count, 6)}|")

Morts dans ce modèle : 983.422, budget utilisé : 72480212.923€ (72.480%), vaccins administrés : 4234862.0
|7|52927.0|
|14|52927.0|
|21|71352.660309|
|28|63471.059489|
|35|77540.0|
|42|89285.059489|
|49|89058.059489|
|56|92927.0|
|63|72927.0|
|70|72927.0|
|77|72927.0|
|84|72927.0|
|91|52927.0|
|98|52927.0|
|105|52927.0|
|112|52927.0|


#### Génération de graphes d'analyse du modèle

In [8]:
x = [day for day,_ in deliveries]
x2 = range(len(x))
n_fig = 0
for p in prov:
    plt.figure(n_fig, figsize=(20,20))
    plt.plot(x2, [max_delivery[p] for _ in x])
    for t in x:
        plt.bar(f"{t+1}", round(vaccin_delivered[p][t+1].x,6))
    plt.title(f"Livraisons {p}")
    plt.savefig(f"figs\\livraison_{p}.png".replace(" ", "_").replace('’', '_').lower())
    # plt.show() # Décommenter pour voir s'afficher les graphes au fur et à mesure des calculs 
    plt.close()
    n_fig += 1

In [9]:
x = range(nb_days)
n_fig = 0
for p in prov:
    plt.figure(n_fig, figsize=(20,20))
    plt.plot(x, [max_stock[p] for _ in x])
    plt.plot(x, [stock[p][day] for day in x])
    plt.title(f"Stock {p}")
    plt.savefig(f"figs\\stock_{p}.png".replace(" ", "_").replace('’', '_').lower())
    # plt.show() # Décommenter pour voir s'afficher les graphes au fur et à mesure des calculs
    plt.close()
    n_fig += 1

In [10]:
x = range(nb_days)
n_fig = 0
for p in prov:
    plt.figure(n_fig, figsize=(20,20))
    plt.plot(x, [max_vaccination[p] for _ in x])
    plt.plot(x, [vaccine_given_per_day[p][t+1] for t in x])
    plt.title(f"Vaccins {p}")
    plt.savefig(f"figs\\vacc_{p}.png".replace(" ", "_").replace('’', '_').lower())
    # plt.show() # Décommenter pour voir s'afficher les graphes au fur et à mesure des calculs
    plt.close()
    n_fig += 1

### Question I.3
#### (a) Estimez l'impact d'une augmentation du budget total.
Comme les budgets ne sont pas utilisé au maximum, l'augmentation du budget n'aurait pas d'influence à priori
#### (b) Estimez l'impact de la disponibilité de doses supplémentaire sur la mortalité. Plus spécifquement, donnez une estimation, potentiellement différente pour chaque jour, de la quantité de doses supplémentaires qui serait nécessaire pour éviter un décès. Commentez.
Comme toutes les doses des jours 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105 et 112 sont toutes délivrées, augmenter le nombre de doses disponibles délivrées ces jours-là aurait un effet positif, car on peut encore livrer puis stocker ou utiliser au total :

|Jour |Doses manquantes|
--- | ---
|7|52927.0|
|14|52927.0|
|21|71352.660309|
|28|63471.059489|
|35|77540.0|
|42|89285.059489|
|49|89058.059489|
|56|92927.0|
|63|72927.0|
|70|72927.0|
|77|72927.0|
|84|72927.0|
|91|52927.0|
|98|52927.0|
|105|52927.0|
|112|52927.0|

#### (c) Estimez l'impact d'une augmentation des différentes capacités maximales présentes dans le problème. A nouveau vos réponses seront exprimée en termes d'augmentations nécessaires pour éviter un décès. Commentez.


### Question I.4 Sans les implémenter, expliquez comment vous pourriez ajouter les aspects suivants à votre modèle (tout en le gardant linéaire) :
#### (a) Possibilité d'acheter des doses supplémentaires : une certaine quantité de doses supplémentaires Q est disponible chaque jour, dont la moitié à un prix unitaire P1 et l'autre moitié à un prix unitaire P2 supérieur à P1.
Rajouter une variable supplémentaire (le maximum de la différence entre le nombre de doses  qui représenterait les doses au prix P2 qui feraient augmenter le coût total d'un montant P2-P1.
#### (b) Après la fin de la campagne de vaccination le modèle proposé continue de prédire des contaminations. Comment intégrer au modèle l'estimation du nombre de décès qui résultera sur cette période (sur un horizon de temps potentiellement infini)
Il suffirait de reprendre les chiffres des personnes encore susceptibles de tomber malades à la fin de la campagne de vaccination, et de prévoir les morts au fur et à mesure de l'avancement des jours, jusqu'à ce que tout le monde soit tombé malade, ou alors forcer les vaccins donnés après la fin de la campagne à 0, et de continuer la simulation jusqu'à ce que il n'y ai plus personne de susceptible de tomber malade

### Question I.5